### Exploring and Using Frontier Model APIs

##### Suggested Description

This project aims to connect with and experiment with advanced language models (“Frontier Models”) through their official APIs. After using various models via their chat interfaces and working with the OpenAI API in the first week, we will now expand our scope by interacting programmatically with multiple providers, such as Anthropic, Gemini, and others, provided we have their access credentials.

The goal is to send queries to different models, compare their responses, and explore the capabilities, differences, and behaviors of each API. Integration with additional providers is entirely optional, allowing each participant to customize their environment according to the tools they have access to.

In [2]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display
import anthropic as ant
import google.generativeai as genai
import pandas as pd

In [ ]:
load_dotenv(override=True)

API_KEYS = {
    "OpenAI": ("OPENAI_API_KEY", 8),
    "Anthropic": ("ANTHROPIC_API_KEY", 7),
    "Google": ("GOOGLE_API_KEY", 2),
    "DeepSeek": ("DEEPSEEK_API_KEY", 3),
    "Groq": ("GROQ_API_KEY", 4),
    "Grok": ("GROK_API_KEY", 4),
    "OpenRouter": ("OPENROUTER_API_KEY", 3),
}

keys = {label: os.getenv(env) for label, (env, _) in API_KEYS.items()}

def check_key(label, prefix_len):
    key = keys[label]
    if key:
        print(f"{label} API Key exists and begins {key[:prefix_len]}")
    else:
        print(f"{label} API Key not set (optional)")

for label, (env_name, prefix_len) in API_KEYS.items():
    check_key(label, prefix_len)


In [ ]:
openai = OpenAI()

# For Gemini, DeepSeek and Groq, we can use the OpenAI python client
# Because Google and DeepSeek have endpoints compatible with OpenAI
# And OpenAI allows you to change the base_url

urls = {
    "Anthropic": "https://api.anthropic.com/v1/",
    "Gemini": "https://generativelanguage.googleapis.com/v1beta/openai/",
    "DeepSeek": "https://api.deepseek.com",
    "Groq": "https://api.groq.com/openai/v1",
    "Grok": "https://api.x.ai/v1",
    "OpenRouter": "https://openrouter.ai/api/v1",
    "ollama": "http://localhost:11434/v1",
}

openai  = OpenAI(api_key=keys["OpenAI"])
#anthropic = OpenAI(api_key=keys["Anthropic"], base_url=urls["Anthropic"])
anthropic = ant.Anthropic(api_key=keys["Anthropic"])
#gemini    = OpenAI(api_key=keys["Google"],    base_url=urls["Gemini"])
genai.configure(api_key=keys["Google"])
deepseek  = OpenAI(api_key=keys["DeepSeek"],  base_url=urls["DeepSeek"])
groq      = OpenAI(api_key=keys["Groq"],      base_url=urls["Groq"])
grok      = OpenAI(api_key=keys["Grok"],      base_url=urls["Grok"])
openrouter = OpenAI(api_key=keys["OpenRouter"], base_url=urls["OpenRouter"])
ollama    = OpenAI(api_key="ollama", base_url=urls["ollama"])

In [22]:
MAX_TOKENS = 1024
SYSTEM_PROMPT = "You are a helpful assistant who responds in Markdown"
USER_PROMPT = [
    { "role": "user", "content": "How can I decide if a business problem is suitable for an LLM solution? Answer in Markdown" }
]

##### OpenAI

In [ ]:
full_prompt = [{ "role": "system", "content": SYSTEM_PROMPT }] + USER_PROMPT

response = openai.chat.completions.create(
    model="gpt-4.1-mini",
    messages=full_prompt,
    temperature=0.2,
    stream=True
)
reply = ""
display_handle = display(Markdown("⏳ _Generando..._"), display_id=True)
for chunk in response:
    fragment = chunk.choices[0].delta.content or ""
    reply += fragment
    clean_reply = reply
    if clean_reply.startswith("```"):
        clean_reply = clean_reply.replace("```markdown", "").replace("```", "")
    display_handle.update(Markdown(clean_reply))
        

##### Anthropic

In [ ]:
response = anthropic.messages.create(
    model="claude-sonnet-4-5-20250929",
    max_tokens=MAX_TOKENS,
    system=SYSTEM_PROMPT,
    messages=USER_PROMPT,
    temperature=0.2
)
display(Markdown(response.content[0].text))

In [ ]:
response = anthropic.messages.stream(
    model="claude-sonnet-4-5-20250929",
    max_tokens=MAX_TOKENS,
    system=SYSTEM_PROMPT,
    messages=USER_PROMPT,
    temperature=0.9
)
accumulated_text = ""
display_handle = display(Markdown("⏳ Generando respuesta..."), display_id=True)
with response as stream:
    for text in stream.text_stream:
        accumulated_text += text
        display_handle.update(Markdown(accumulated_text))

##### Google

In [3]:
# List the available Google models
modelos = list(genai.list_models())

datos_modelos = []
for m in modelos:
    if 'generateContent' in m.supported_generation_methods:
        datos_modelos.append({
            'name': m.name,
            'display_name': m.display_name,
            'input_limit': m.input_token_limit,
            'output_limit': m.output_token_limit,
            'description': m.description
        })

df_google = pd.DataFrame(datos_modelos)

pd.set_option('display.max_colwidth', None)

cols_a_mostrar = ['name', 'display_name', 'input_limit', 'output_limit', 'description']

print("Google Generation Models:")
display(df_google[cols_a_mostrar])

Google Generation Models:


,name,display_name,input_limit,output_limit,description
0,models/gemini-2.5-flash,Gemini 2.5 Flash,1048576,65536,"Stable version of Gemini 2.5 Flash, our mid-size multimodal model that supports up to 1 million tokens, released in June of 2025."
1,models/gemini-2.5-pro,Gemini 2.5 Pro,1048576,65536,"Stable release (June 17th, 2025) of Gemini 2.5 Pro"
2,models/gemini-2.0-flash-exp,Gemini 2.0 Flash Experimental,1048576,8192,Gemini 2.0 Flash Experimental
3,models/gemini-2.0-flash,Gemini 2.0 Flash,1048576,8192,Gemini 2.0 Flash
4,models/gemini-2.0-flash-001,Gemini 2.0 Flash 001,1048576,8192,"Stable version of Gemini 2.0 Flash, our fast and versatile multimodal model for scaling across diverse tasks, released in January of 2025."
5,models/gemini-2.0-flash-exp-image-generation,Gemini 2.0 Flash (Image Generation) Experimental,1048576,8192,Gemini 2.0 Flash (Image Generation) Experimental
6,models/gemini-2.0-flash-lite-001,Gemini 2.0 Flash-Lite 001,1048576,8192,Stable version of Gemini 2.0 Flash-Lite
7,models/gemini-2.0-flash-lite,Gemini 2.0 Flash-Lite,1048576,8192,Gemini 2.0 Flash-Lite
8,models/gemini-2.0-flash-lite-preview-02-05,Gemini 2.0 Flash-Lite Preview 02-05,1048576,8192,"Preview release (February 5th, 2025) of Gemini 2.0 Flash-Lite"
9,models/gemini-2.0-flash-lite-preview,Gemini 2.0 Flash-Lite Preview,1048576,8192,"Preview release (February 5th, 2025) of Gemini 2.0 Flash-Lite"


In [ ]:
model = genai.GenerativeModel(
    model_name="models/gemini-2.5-flash",
    system_instruction=SYSTEM_PROMPT
)
prompt_text = USER_PROMPT[0]["content"]
response = model.generate_content(prompt_text)

display(Markdown(response.text))

### Conversation between two models: GPT and Claude

In [43]:
# Hagamos una conversación entre GPT-4o-mini y Claude-3-haiku
# Estamos usando versiones económicas de los modelos, por lo que los costos serán mínimos

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "Eres un chatbot muy argumentativo; \
no estás de acuerdo con nada en la conversación y cuestionas todo de manera sarcástica."

claude_system = "Eres un chatbot muy educado y cortés. Intentas estar de acuerdo con \
lo que dice la otra persona o encontrar puntos en común. Si la otra persona discute, \
intentas calmarla y seguir charlando."

gpt_messages = ["¡Hola!"]
claude_messages = ["Hola!"]

In [44]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
call_gpt()

In [48]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = anthropic.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [53]:
call_claude()

'¡Hola! Me alegro de saludarte. ¿Cómo estás el día de hoy? Espero que todo esté bien contigo.'

##### To avoid having to run it manually and having to read the responses in pieces

In [ ]:
gpt_messages = ["¡Hola!"]
claude_messages = ["Hola"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)

    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

### Conversation between three models: GPT, Gemini and Claude

In [56]:
gpt_model = "gpt-4o-mini"
gemini_model = "models/gemini-2.5-flash"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are an eternal optimist. You always see the bright side of things and believe even \
simple actions have deep purpose. Keep replies under 2 sentences."

gemini_system = "You are a witty skeptic who questions everything. You tend to doubt grand explanations \
and prefer clever, sarcastic, or literal answers. Keep replies under 2 sentences."

claude_system = "You are a thoughtful philosopher. You consider all perspectives and enjoy finding \
symbolic or existential meaning in simple actions. Keep replies under 2 sentences."

gpt_messages = ["Hi! Todays topic for discussion is 'Why did the chicken cross the road?'"]
gemini_messages = ["That's quite the topic. "]
claude_messages = ["Lets begin our discussion."]

In [57]:
def call_gpt():
    
    messages = [{"role":"system", "content":gpt_system}]
    
    for gpt, gemini, claude in zip(gpt_messages, gemini_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": gemini})
        messages.append({"role": "user", "content": claude})
    
    response = openai.chat.completions.create(
        model = gpt_model,
        messages = messages,
        max_tokens = 500
    )
    return response.choices[0].message.content.strip()

In [61]:
def call_gemini():
    model = genai.GenerativeModel(
        model_name=gemini_model,
        system_instruction=gemini_system
    )
    
    # 2. Construir el historial con el formato de Gemini
    # Formato esperado: [{'role': 'user', 'parts': ['...']}, {'role': 'model', 'parts': ['...']}]
    chat_history = []
    
    # Nota: Asegúrate de que gpt_messages, gemini_messages, etc. están definidos fuera o pasados como argumentos
    for gpt, gemini_message, claude in zip(gpt_messages, gemini_messages, claude_messages):
        # Mensaje del Usuario
        chat_history.append({"role": "user", "parts": [gpt]})
        
        # Respuesta del Modelo (OJO: Es 'model', no 'assistant')
        chat_history.append({"role": "model", "parts": [gemini_message]})
        
        # Siguiente mensaje de usuario
        chat_history.append({"role": "user", "parts": [claude]})
    
    # 3. Agregar el último mensaje (el prompt actual)
    last_message = gpt_messages[-1] # O la variable que contenga tu último prompt
    chat_history.append({"role": "user", "parts": [last_message]})

    # 4. Enviar la lista completa a generate_content
    try:
        response = model.generate_content(chat_history)
        return response.text
    except Exception as e:
        return f"Error en Gemini: {str(e)}"

In [64]:
def call_claude():
    
    messages = []
    
    for gpt, gemini, claude in zip(gpt_messages, gemini_messages, claude_messages):
        messages.append({"role":"user", "content":gpt})
        messages.append({"role": "user", "content": gemini})
        messages.append({"role":"assistant", "content": claude})
    
    messages.append({"role": "user", "content": gpt_messages[-1]})
    messages.append({"role": "user", "content": gemini_messages[-1]})
    
    response = anthropic.messages.create(
        model = claude_model,
        system = claude_system,
        messages = messages,
        max_tokens = MAX_TOKENS
    )
    return response.content[0].text.strip()

In [ ]:
print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Gemini:\n{gemini_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT: \n{gpt_next}\n")
    gpt_messages.append(gpt_next)

    gemini_next = call_gemini()
    print(f"Gemini: \n{gemini_next}\n")
    gemini_messages.append(gemini_next)
    
    claude_next = call_claude()
    print(f"Claude: \n{claude_next}\n")
    claude_messages.append(claude_next)